In [ ]:
ALGORITHM: P2P_Bill_Sharing_Billing
# References: Bill sharing formulas & community aggregates from main p2p.pdf. :contentReference[oaicite:2]{index=2}
# Structure & time-slot conventions inspired by p2p_sdr_algorithm.pdf. :contentReference[oaicite:3]{index=3}

INPUTS:
  H                  # number of time slots (e.g., 96 for 15-min slots)
  N_prosumers        # list/array of prosumers (index 1..N)
  dt                 # duration of each slot in hours (e.g., 0.25 for 15-min)
  lambda_buy_grid    # c_BFG  (grid buy price, price to buy from main grid)
  lambda_sell_grid   # c_STG  (grid sell price, feed-in tariff)
  For each prosumer i:
    L[i][h]          # demand (load) of prosumer i at slot h (kW)
    PV[i][h]         # PV generation of prosumer i at slot h (kW)

OUTPUTS:
  For each prosumer i:
    C[i]             # final bill (monetary) for the billing horizon
  Community aggregates:
    E_im_total, E_ex_total, E_BFG, E_STG
    internal_buy_price c_im, internal_sell_price c_ex   # community-level prices

# --- INITIALIZATION ---
For i in 1..N:
  Initialize C[i] = 0
Initialize arrays:
  M_i[h] = 0         # overlapping generation consumed by same household
  P_im_i[h] = 0      # import (residual demand) per household per slot
  P_ex_i[h] = 0      # export (surplus) per household per slot
  sum_P_im_total[h] = 0
  sum_P_ex_total[h] = 0

# --- STEP 1: Per-slot household accounting (instantaneous) ---
For h in 1..H:
  For each prosumer i in 1..N:
    # instantaneous overlap per eq (1): M_n(t) = min(L_n(t), PV_n(t))
    M_i[h] = min( L[i][h], PV[i][h] )

    # residual import and export per eq (2),(3)
    P_im_i[h] = L[i][h] - M_i[h]         # >= 0 (amount household needs from outside)
    P_ex_i[h] = PV[i][h] - M_i[h]        # >= 0 (amount household can export)

    # accumulate community totals for this slot
    sum_P_im_total[h]  += P_im_i[h]
    sum_P_ex_total[h]  += P_ex_i[h]

# --- STEP 2: Compute energy totals over horizon per eq (4),(5),(10),(11) ---
E_im_total = 0
E_ex_total = 0
For h in 1..H:
  E_im_total += sum_P_im_total[h] * dt       # total kWh imported by individuals
  E_ex_total += sum_P_ex_total[h] * dt       # total kWh exported by individuals